# SMTM's TPS, Sep-2021 EDA

"SHOW ME THE MEDAL", My Team. We scored 0.81785 with ideas shared in the 'Code' and 'Discussion' (including Team GMO). In this note, We share only EDA note and will share ideas of preprocessing and models. We studied some of EDA notebooks below.

TPS September 2021 EDA :  https://www.kaggle.com/dwin183287/tps-september-2021-eda      
CATBOOST BRONZE :  https://www.kaggle.com/lonnieqin/catboost-tabular-playground-prediction-sep-2021       
TPS Soft Voting (XGB/CB/LGBM) :  https://www.kaggle.com/dmitryuarov/tps-soft-voting-xgb-cb-lgbm#Basic-informationn        
Team GMO's EDA :   https://www.kaggle.com/seoltommy/gmo-submission

My team used Google Colab, but writing this note for sharing. Reading this note, please consider for my writing skills in English. For these several years, I have been apart from English. By the way, I will start this EDA note, and hope this would be helpful. 

### Import

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as sk

# for fast visualization
%matplotlib inline

#  retina display 
%config InlineBackend.figure_format = 'retina'

# for minus
mpl.rc('axes', unicode_minus=False)

### Load Dataset
- One of kaggle Notbook's benefits is easy for loading datas as you know. 

In [ ]:
# train data
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
# test data
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
# sample solution
ss = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
# let's look into train data
print(train.shape)
train.describe()

In [ ]:
# let's look into train data
print(test.shape)
test.describe()

### Overview on null values

In [ ]:
# train data
print("=====Train Data - null values in Columns=====", end = '\n\n')
# Column
print("Counts of null values in a Column")
print( train.isnull().sum(), end = '\n\n')

print("Percentage of null values in a Column")
print( round( 100*train.isnull().sum()/len(train), 2), end='\n\n' )

print("Average Percentage of null values in a Column")
print( round(np.array(100*train.isnull().sum()/len(train)).mean(),2))

In [ ]:
# train data
print("=====Train Data - null values in Rows=====", end = '\n\n')
# Row
print("Counts of null values in a Row")
print( train.isnull().sum(axis =1), end = '\n\n')
print()

print("Percentage of Counts of null values in a Row")
print( round( 100*train.isnull().sum(axis =1)/(train.shape[1]), 2))

In [ ]:
# test data
print("=====Test Data - null valuesin Columns=====", end = '\n\n')
# Column 
print("Counts of null values in a Column")
print(test.isnull().sum(), end = '\n\n')

print("Percentage of Counts of null values in a Column")
print(round(100*test.isnull().sum()/(test.shape[0]), 2), end='\n\n')

print("Average Percentage of null values in a Column")
print(round(np.array(100*test.isnull().sum()/(test.shape[0])).mean(),2))

In [ ]:
# test data
print("=====Test Data - null values in Rows=====", end = '\n\n')
# Row 
print("Counts of null values in a Row")
print( test.isnull().sum(axis =1), end = '\n\n')

print("Percentage of Counts of null values in a Row")
print(round( 100*test.isnull().sum(axis =1)/(train.shape[1]),2))

### Let's Look into 'null values'
One of EDA nots we looked into says 'null values in each row' is related to 'claim' feature. You can see this insight in Discussion. Therefore, my team decided to look into null values further and further. 

In [ ]:
# null values in column
train.isnull().sum(axis = 0)

In [ ]:
# null values in rows
train.isnull().sum(axis = 1)

## column 'miss_cnt'
For further study, we also made new column, 'miss_cnt', which shows counts of null values in each row. 

In [ ]:
train['miss_cnt'] = train.isnull().sum(axis =1)
train.miss_cnt

### We should do same thing on test data.

In [ ]:
test['miss_cnt'] = test.isnull().sum(axis =1)
test.miss_cnt

## miss_train, no_miss_train
We made 'miss_train' pandas DataFrame, which contains rows that have more than one null value in a row and made 'no_miss_train' pandas DataFrame, which contains rows that have more no null value in a row. 

In [ ]:
# miss_train

miss_train = train.loc[train.isnull().sum(axis =1) != 0, :]
print(miss_train.shape)
miss_train.head(3)

In [ ]:
# no_miss_train

no_miss_train = train.loc[train.isnull().sum(axis=1) == 0, :]
print(no_miss_train.shape)
no_miss_train.head(3)

#### Checked that there is no data which we have missed. 

In [ ]:
miss_train.shape[0] + no_miss_train.shape[0] == train.shape[0]

## Visulalize Train & Test
 Checked distribution of each feature in train data and test data. We also used histplot of seaborn's with kde option.    
 We discriminate between both's distributions by giving two different line colors.

In [ ]:
fig = plt.figure(figsize = (15, 71)) 
cols = train.columns.tolist()[1:119]
for num in cols:
    plt.subplot(24, 5, cols.index(num) +1 )
    sns.set_style("white") 
    plt.title(num, size = 12)
    a = sns.histplot(data = train, x =num, color = '#f9ba32', linewidth = 1.3, kde=True)
    sns.histplot(data = test, x=num, color = '#426e86', linewidth = 1.3,kde=True)
    plt.ylabel('')
    plt.xlabel('') 
    
    plt.xticks([])
    plt.yticks([])
    for lc in ['right', 'left', 'top']:
        a.spines[lc].set_visible(False)
        a.spines['bottom'].set_linewidth(1.2)

fig.tight_layout(h_pad = 3)

plt.figtext(0.335, 1.02, 'Distribution of features', color = '#2f3131', size = 25) # moonspace 폰트 좋아하지만, 코랩에 없으므로 생략. 
plt.figtext(0.3, 1.01, 'train', color = '#f9ba32', size = 19)
plt.figtext(0.66, 1.01, 'test', color = '#426e86', size = 19)

plt.show()

## Same Distribution shape and different Scale
You can see that distribution shape is the same between train and test, just both seem different in scale.

## Visulalize distribution between Train data 'claim' is 0 and 'claim' is 1. 
We feel needs to check distribution of Train data when train's column 'claim'(target column) is 0 and Train data when train's column 'claim' is 1, after looking above hisplots above. We just did this in the same way.

In [ ]:
fig = plt.figure(figsize = (15, 71)) 
cols = train.columns.tolist()[1:119]
for num in cols:
    plt.subplot(24, 5, cols.index(num) +1 )
    sns.set_style("white") 
    plt.title(num, size = 12)

    a = sns.histplot(data= train.loc[train.claim == 0], x = num, color = '#f9ba32', linewidth = 1.3, kde=True)
    sns.histplot(data= train.loc[train.claim == 1], x = num, color = '#426e86', linewidth = 1.3, kde=True)
    plt.ylabel('')
    plt.xlabel('') 
    plt.xticks([])
    plt.yticks([])
    for lc in ['right', 'left', 'top']:
        a.spines[lc].set_visible(False)
        a.spines['bottom'].set_linewidth(1.2)

fig.tight_layout(h_pad = 3)

plt.figtext(0.335, 1.02, 'Distribution of features Whether train.claim == 0 or 1', color = '#2f3131', size = 25) 
plt.figtext(0.3, 1.01, 'No claim', color = '#f9ba32', size = 19) 
plt.figtext(0.66, 1.01, 'Claimed', color = '#426e86', size = 19)

plt.show()

## Almost same distribution
Every feature's distributions between them are almost same and even scale is almost same, too.





## How about miss_train?
Visulalized distribution between 'miss_train' data's column 'claim' is 0 and 'claim' is 1. 

In [ ]:
fig = plt.figure(figsize = (15, 71)) 
cols = train.columns.tolist()[1:119]
for num in cols:
    plt.subplot(24, 5, cols.index(num) +1 )
    sns.set_style("white") # 이쁜 white
    plt.title(num, size = 12)

    a = sns.histplot(data= miss_train.loc[miss_train.claim == 0], x = num, color = '#f9ba32', linewidth = 1.3, kde=True)
    sns.histplot(data= miss_train.loc[miss_train.claim == 1], x = num, color = '#426e86', linewidth = 1.3, kde=True)
    plt.ylabel('')
    plt.xlabel('') 
    plt.xticks([])
    plt.yticks([])
    for lc in ['right', 'left', 'top']:
        a.spines[lc].set_visible(False)
        a.spines['bottom'].set_linewidth(1.2)

fig.tight_layout(h_pad = 3)

plt.figtext(0.335, 1.02, 'miss_train - Distribution of features Whether claim == 0 or 1', color = '#2f3131', size = 25) 
plt.figtext(0.3, 1.01, 'No claim', color = '#f9ba32', size = 19) 
plt.figtext(0.66, 1.01, 'Claimed', color = '#426e86', size = 19)

plt.show()

### Difference in scale
Still same distribution, but difference in scale is more clearer!








## And no_miss_train?
Visulalized distribution between 'no_miss_train' data's column 'claim' is 0 and 'claim' is 1. 

In [ ]:
fig = plt.figure(figsize = (15, 71)) 
cols = train.columns.tolist()[1:119]
for num in cols:
    plt.subplot(24, 5, cols.index(num) +1 )
    sns.set_style("white") 
    plt.title(num, size = 12)

    a = sns.histplot(data= no_miss_train.loc[no_miss_train.claim == 0], x = num, color = '#f9ba32', linewidth = 1.3, kde=True)
    sns.histplot(data= no_miss_train.loc[no_miss_train.claim == 1], x = num, color = '#426e86', linewidth = 1.3, kde=True)
    plt.ylabel('')
    plt.xlabel('') 
    plt.xticks([])
    plt.yticks([])
    for lc in ['right', 'left', 'top']:
        a.spines[lc].set_visible(False)
        a.spines['bottom'].set_linewidth(1.2)

fig.tight_layout(h_pad = 3)

plt.figtext(0.335, 1.02, 'No miss_train - Distribution of features Whether claim == 0 or 1', color = '#2f3131', size = 25) 
plt.figtext(0.3, 1.01, 'No claim', color = '#f9ba32', size = 19) 
plt.figtext(0.66, 1.01, 'Claimed', color = '#426e86', size = 19)

plt.show()

### Difference in scale
Still same distribution, difference in scale got clearer. However, scale in 'Claimed' is more larger while above's is more smaller. This can be helpful if we can input this insight into ML. 













## Lastly, Target column 'claim'

In [ ]:
# target value visualizing
# referenced: https://wikidocs.net/92114

plt.figure(figsize = (9, 9))
plt.title('Target Variable Counts', fontsize = 29)
plt.pie(train.claim.value_counts(), labels=['No Claim', 'Claim'], textprops = {'fontsize':20},
                                    shadow =True, startangle = 120, counterclock = False, 
                                    explode=[0.10, 0], colors =['#ff9999', '#ffc000'])
plt.show()

## How to treat null values and 'miss_cnt'?

Visualize null values with seaborn's countplot in miss_train data and no_miss_train data first.

In [ ]:
# miss_train - miss_cnt
plt.figure(figsize=(15, 9))
plt.title("miss_train - hue: claim - miss_cnt", fontsize= 16)
sns.countplot(data=miss_train, x='miss_cnt', hue='claim')

In [ ]:
# no_miss_train - miss_cnt
plt.figure(figsize=(15, 9))
plt.title("no_miss_train - hue: claim - miss_cnt", fontsize= 16)
sns.countplot(data=no_miss_train, x='miss_cnt', hue='claim')

### Relation between number of null values and 'claim' columns.

In [ ]:
miss_cnt_amount_claim =miss_train.groupby(miss_train.miss_cnt)['claim'].count()
miss_cnt_amount_claim 

In [ ]:
miss_cnt_claim = pd.pivot_table(data=miss_train, index='miss_cnt', values='claim', aggfunc=np.sum)
miss_cnt_claim

In [ ]:
miss_cnt_ratio = pd.concat([miss_cnt_claim, miss_cnt_amount_claim], axis = 1 )
miss_cnt_ratio.columns = ['claim_counts', 'total_amount']
miss_cnt_ratio['percantage'] = miss_cnt_ratio.claim_counts / miss_cnt_ratio.total_amount
miss_cnt_ratio

### Strong relation between the number of null values and claim. 
we can transform this into categorical feature. You can check that a row data with 2 ~ 13 null values has  70 ~ 80% probability to claim and a row data with 1 or 14 null values has 50% probability to claim. My team made new categorical feature, 'mc_cat', whose categorical values are 0, 1, 2 on train data and test data by following rules. 

- 2 ~ 13 null values -> 2
- 1, 14 null values -> 1
- no null values -> 0 



In [ ]:
def mc_cat(val):
    if val == 0:
        return 0
    elif val == 1 or val == 14:
        return 1
    else:
        return 2

In [ ]:
train['mc_cat'] = train['miss_cnt'].apply(mc_cat)
test['mc_cat'] = test['miss_cnt'].apply(mc_cat)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train[['miss_cnt', 'mc_cat']]

In [ ]:
test[['miss_cnt', 'mc_cat']]

## IQR
It is important how to treat outliers. Therefore, we extracted row-index of outliers in each feature and then counted the number of features in which one row could be an outlier data. And visualized this.

In [ ]:
def get_outlier(df, column, weight=1.6):
    quantile_25 = df[column].quantile(0.25)
    quantile_75 = df[column].quantile(0.75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
  
    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index

    return outlier_idx

### New column 'out_cnt'
- 'out_cnt' for counting features in which one row could be an outlier data per row. Created this column in both train data and test data.
- used 'for' in python for counting features from f1 to f118 and plus 1 whenever a row data becomes an outlier in each feature. 

In [ ]:
train['out_cnt'] =0
for col in train.columns[1:119]:
    outlier_idx = get_outlier(train, col)
    train.loc[outlier_idx, 'out_cnt'] += 1

In [ ]:
train['out_cnt']

In [ ]:
test['out_cnt'] = 0
for col in test.columns[1:119]:
    outlier_idx = get_outlier(test, col)
    test.loc[outlier_idx, 'out_cnt'] += 1

In [ ]:
test['out_cnt']

#### Visualized about Counts of Outliers in rows
Used Seaborn's countplot for visualizing values of 'out_cnt'.

In [ ]:
# Visualizing train data
plt.figure(figsize=(12,8))
sns.countplot(data=train, x='out_cnt', hue='claim')

In [ ]:
# Visualizing test data
plt.figure(figsize=(12,8))
sns.countplot(data=test, x='out_cnt')

### RobustScaler instead of dropping outliers
We concluded that it is better to using RobustScaler instead of dropping outliers. It could be dangerous to drop outliers(row datas) becuase amount of outliers for drop is large. 

In [ ]:
print(test.loc[test.out_cnt >= 9].shape[0])

### Seperate numeric columns and categorical columns

In [ ]:
X_num = train.iloc[:, 1:119]
test_num = test.iloc[:, 1:119]

In [ ]:
X_num.describe()

In [ ]:
test_num.describe()

### GMO's EDA note.
This is why I seperated numerical columns and categorical columns. I found two things interesting and important from Team GMO's EDA note (https://www.kaggle.com/seoltommy/gmo-submission).  

- How to impute missing data(NaN)    
 They looked into every column closely, and they imputed missing values with aporopriate value to each feature.

- Column 'multiply'     
 I had no idea about this before checked their note. They said this feature was helpful in ML predicts. 



#### Check these below!
 



In [ ]:
# Source: https://www.kaggle.com/seoltommy/gmo-submission).

# Data preprocessing functions
def get_stats_per_row(data):
    features = [x for x in data.columns.values if x[0]=="f"] 
    
    data['max_row'] = data[features].max(axis=1)
    data['min_row'] = data[features].min(axis=1)
    data['std'] = data[features].std(axis=1)
    # If the difference between the mean and the median was large, the median function was used.
    # If the difference is small the quantile function was used.
    # Quantile function has better than mean function.
    median_set = ['f9', 'f12', 'f26', 'f27', 'f28', 'f32', 'f33', 'f35', 'f62', 'f74', 'f82', 'f86', 'f98', 'f108', 'f116']
    for col_name in features:
        if col_name in median_set:
            data[col_name].fillna(data[col_name].median(), inplace=True)
        else:
            data[col_name].fillna(data[col_name].quantile(0.75), inplace=True)
            
    # Multiply feature is the feature that multiply all values in a row
    data['multiply'] = 1
    for feature in features:
        data['multiply'] = data[feature] * data['multiply']
    
    return data


In [ ]:
X_num = get_stats_per_row(X_num)
test_num = get_stats_per_row(test_num)

In [ ]:
X_num.describe()

In [ ]:
test_num.describe()